In [ ]:
%load_ext jupyter_black

In [ ]:
from datetime import datetime, timedelta
from IPython.display import display, HTML
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pytz
import random
import threading
from time import sleep, time

from utils import (
    calculate_pnl,
    get_quantiles,
    interpolate_red_green,
    num_decimals,
    plot_bars,
    plot_line,
    plot_quantiles,
    plot_table,
)

In [ ]:
# Constants

In [ ]:
DATA_DIR = "../trades/"
LOG_DIR = "../logs/"

REFRESH_SEC = 1
NUM_TOP_TRADERS = 10
NUM_RECENT_TRADES = 20

FILENAME = ""
START_DATETIME = ""
END_DATETIME = ""
ADDRESS = ""
PRICE_PER_TICK = 0.0001
UNITS_PER_LOT = 0.01

In [ ]:
DATE_ENTRY_LABEL_WIDTH = 100
DATE_ENTRY_WIDTH = 400

TOKEN_ENTRY_LABEL_WIDTH = 100
TOKEN_ENTRY_WIDTH = 225
REFRESH_BUTTON_WIDTH = 225

LOG_WIDTH = 500

SUMMARY_LABEL_SIZE = 15
SUMMARY_VALUE_SIZE = 45

CHART_WIDTH = 585
CHART_HEIGHT = 300
CHART_PADDING = 5
CHART_MARGIN = 1

RECENT_TRADES_WIDTH = 1198
RECENT_TRADES_HEIGHT = 400

In [ ]:
# Widgets

In [ ]:
chart_layout = {
    "border": "1px solid black",
    "padding": f"{CHART_PADDING}px",
    "margin": f"{CHART_MARGIN}px",
}

filename_wid = widgets.Text(
    value=FILENAME,
    description="Filename:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
start_time_wid = widgets.DatetimePicker(
    description="Start:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)
end_time_wid = widgets.DatetimePicker(
    description="End:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)
address_wid = widgets.Text(
    value=ADDRESS,
    description="Address:",
    style={"description_width": f"{DATE_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{DATE_ENTRY_WIDTH}px"),
)

price_per_tick_wid = widgets.FloatText(
    value=PRICE_PER_TICK,
    step=PRICE_PER_TICK,
    description="Price per Tick:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
units_per_lot_wid = widgets.FloatText(
    value=UNITS_PER_LOT,
    step=UNITS_PER_LOT,
    description="Units per Lot:",
    style={"description_width": f"{TOKEN_ENTRY_LABEL_WIDTH}px"},
    layout=widgets.Layout(width=f"{TOKEN_ENTRY_WIDTH}px"),
)
refresh_button_wid = widgets.Button(
    description="Refresh", layout=widgets.Layout(width=f"{REFRESH_BUTTON_WIDTH}px")
)


log_out_wid = widgets.Output(layout={"width": f"{LOG_WIDTH}px"})

pnl_out_wid = widgets.Output(layout=chart_layout)
inventory_out_wid = widgets.Output(layout=chart_layout)
size_out_wid = widgets.Output(layout=chart_layout)
top_makers_out_wid = widgets.Output(layout=chart_layout)
summary_out_wid = widgets.Output(layout=chart_layout)
timestamp_delta_out_wid = widgets.Output(layout=chart_layout)
timestamp_delay_out_wid = widgets.Output(layout=chart_layout)
top_takers_out_wid = widgets.Output(layout=chart_layout)

trades_out_wid = widgets.Output()
top_makers_df_out_wid = widgets.Output()
top_takers_df_out_wid = widgets.Output()

In [ ]:
spacer = widgets.Box(layout=widgets.Layout(height="20px"))

time_range_vbox = widgets.VBox(
    [filename_wid, start_time_wid, end_time_wid, address_wid]
)
instrument_vbox = widgets.VBox(
    [price_per_tick_wid, units_per_lot_wid, refresh_button_wid],
)

left_vbox = widgets.VBox(
    [pnl_out_wid, inventory_out_wid, size_out_wid, top_makers_out_wid]
)
right_vbox = widgets.VBox(
    [
        summary_out_wid,
        timestamp_delta_out_wid,
        timestamp_delay_out_wid,
        top_takers_out_wid,
    ]
)

parameters_hbox = widgets.HBox([time_range_vbox, instrument_vbox, log_out_wid])
charts_hbox = widgets.HBox([left_vbox, right_vbox])
makers_takers_hbox = widgets.HBox([top_makers_df_out_wid, top_takers_df_out_wid])

In [ ]:
# Chart Definitions

In [ ]:
pnl_fig = plot_line(
    pd.Series(),
    [pd.Series(), pd.Series()],
    ["PnL", "Break Even"],
    title="PnL",
    show_legend=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

pnl_fig = go.FigureWidget(pnl_fig)

with pnl_out_wid:
    display(pnl_fig)

In [ ]:
inventory_fig = plot_line(
    pd.Series(),
    [pd.Series(), pd.Series()],
    ["Inventory", "Flat"],
    title="Inventory",
    show_legend=False,
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

inventory_fig = go.FigureWidget(inventory_fig)

with inventory_out_wid:
    display(inventory_fig)

In [ ]:
size_fig = plot_quantiles(
    pd.Series(),
    round_decimals=num_decimals(units_per_lot_wid.value),
    bins=50,
    name="Size",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

size_fig = go.FigureWidget(size_fig)

with size_out_wid:
    display(size_fig)

In [ ]:
timestamp_delta_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=50,
    name="Timestamp between Trades (s)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

timestamp_delta_fig = go.FigureWidget(timestamp_delta_fig)

with timestamp_delta_out_wid:
    display(timestamp_delta_fig)

In [ ]:
timestamp_delay_fig = plot_quantiles(
    pd.Series(),
    round_decimals=2,
    bins=50,
    name="Timestamp Delay (s)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

timestamp_delay_fig = go.FigureWidget(timestamp_delay_fig)

with timestamp_delay_out_wid:
    display(timestamp_delay_fig)

In [ ]:
summary_fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}],
        [{"type": "indicator"}, {"type": "indicator"}],
    ],
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={"font": {"size": SUMMARY_VALUE_SIZE}},
        title={"text": "Fills", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=1,
    col=1,
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={"font": {"size": SUMMARY_VALUE_SIZE}},
        title={"text": "PnL/Volume (bps)", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=2,
    col=1,
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={
            "font": {"size": SUMMARY_VALUE_SIZE},
            "valueformat": f".{num_decimals(units_per_lot_wid.value)}f",
        },
        title={"text": "Base Volume", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=1,
    col=2,
)

summary_fig.add_trace(
    go.Indicator(
        mode="number",
        value=0,
        number={"font": {"size": SUMMARY_VALUE_SIZE}, "valueformat": ".2f"},
        title={"text": "Quote Volume", "font": {"size": SUMMARY_LABEL_SIZE}},
    ),
    row=2,
    col=2,
)

summary_fig.update_layout(
    width=CHART_WIDTH, height=CHART_HEIGHT, margin=dict(l=0, r=0, t=20, b=0)
)

summary_fig = go.FigureWidget(summary_fig)

with summary_out_wid:
    display(summary_fig)

In [ ]:
top_makers_fig = plot_bars(
    pd.Series(),
    pd.Series(),
    title="Top Makers (Notional Volume)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

top_makers_fig = go.FigureWidget(top_makers_fig)

with top_makers_out_wid:
    display(top_makers_fig)

In [ ]:
top_takers_fig = plot_bars(
    pd.Series(),
    pd.Series(),
    title="Top Takers (Notional Volume)",
    width=CHART_WIDTH,
    height=CHART_HEIGHT,
)

top_takers_fig = go.FigureWidget(top_takers_fig)

with top_takers_out_wid:
    display(top_takers_fig)

In [ ]:
recent_trades_fig = plot_table(
    pd.DataFrame(),
    [],
    column_widths=[0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
    width=RECENT_TRADES_WIDTH,
    height=RECENT_TRADES_HEIGHT,
)

recent_trades_fig = go.FigureWidget(recent_trades_fig)

with trades_out_wid:
    display(recent_trades_fig)

In [ ]:
# Update Loop

In [ ]:
trades = None


def update_trades():
    global trades

    try:
        data_path = f"{DATA_DIR}{filename_wid.value}.csv"
        trades = pd.read_csv(data_path)

        trades["timestamp"] = pd.to_datetime(trades["timestamp"], unit="s")
        trades["timestamp"] = (
            trades["timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
        )

        trades["local_timestamp"] = pd.to_datetime(trades["local_timestamp"], unit="ms")
        trades["local_timestamp"] = (
            trades["local_timestamp"].dt.tz_localize("UTC").dt.tz_convert("US/Eastern")
        )

        if start_time_wid.value is not None:
            trades = trades[trades["timestamp"] >= start_time_wid.value]
        if end_time_wid.value is not None:
            trades = trades[trades["timestamp"] <= end_time_wid.value]

        trader_address = address_wid.value.strip()
        if trader_address != "":
            trades = trades[
                (trades["maker"] == trader_address)
                | (trades["taker"] == trader_address)
            ]

        trades["timestamp_delay"] = trades["local_timestamp"] - trades["timestamp"]
        trades["timestamp_delay"] = trades["timestamp_delay"].dt.total_seconds()

        trades = trades.sort_values(by="timestamp")

        trades["price"] = trades["price"] * price_per_tick_wid.value
        trades["price"] = trades["price"].round(num_decimals(price_per_tick_wid.value))

        trades["size"] = trades["size"] * units_per_lot_wid.value
        trades["size"] = trades["size"].round(num_decimals(units_per_lot_wid.value))

        trades["notional"] = trades["price"] * trades["size"]

        trades["inventory_change"] = trades.apply(
            lambda row: row["size"] if row["side"] == "buy" else -row["size"], axis=1
        )
        trades["inventory"] = trades["inventory_change"].cumsum()
        trades["inventory"] = trades["inventory"].round(
            num_decimals(units_per_lot_wid.value)
        )

    except Exception as e:
        raise e

    disp_cols = [
        "timestamp",
        "timestamp_delay",
        "slot",
        "side",
        "price",
        "size",
        "inventory",
    ]
    disp_trades = trades[disp_cols].tail(NUM_RECENT_TRADES).iloc[::-1]
    recent_trades_fig.data[0].header = dict(values=disp_cols, align="left")
    recent_trades_fig.data[0].cells = dict(
        values=[disp_trades[c] for c in disp_cols], align="left"
    )


def update_summary_fig():
    if trades is None or len(trades) == 0:
        summary_fig.data[0].value = 0
        summary_fig.data[1].value = 0
        summary_fig.data[2].value = 0
        summary_fig.data[3].value = 0

        return

    quote_volume = trades["notional"].sum()
    base_volume = trades["size"].sum()

    trader_address = address_wid.value.strip()
    if trader_address == "":
        trader_address = None
    pnl_series = calculate_pnl(trades, address=trader_address)

    summary_fig.data[0].value = len(trades)
    summary_fig.data[1].value = round((pnl_series.iloc[-1] / quote_volume) * 10000, 2)
    summary_fig.data[1].number.font.color = interpolate_red_green(
        (pnl_series.iloc[-1] / quote_volume) * 10000, -10, 10
    )
    summary_fig.data[2].value = round(
        base_volume, num_decimals(units_per_lot_wid.value)
    )
    summary_fig.data[
        2
    ].number.valueformat = f".{num_decimals(units_per_lot_wid.value)}f"
    summary_fig.data[3].value = round(quote_volume, 2)


def update_line(fig, x_series, y_series_lst):
    for i in range(len(y_series_lst)):
        fig.data[i].x = x_series
        fig.data[i].y = y_series_lst[i]


def update_pnl_fig():
    pnl_series = calculate_pnl(trades)
    flat_pnl_series = pd.Series(0, index=pnl_series.index)

    update_line(pnl_fig, trades["timestamp"], [pnl_series, flat_pnl_series])


def update_inventory_fig():
    inventory_series = trades.set_index("timestamp")["inventory"]
    flat_inventory_series = pd.Series(0, index=inventory_series.index)

    update_line(
        inventory_fig, trades["timestamp"], [inventory_series, flat_inventory_series]
    )


def update_quantiles(fig, series, name, round_decimals):
    quantiles = get_quantiles(series, name)

    table_header = dict(values=list(quantiles.columns), align="left")
    table_cells = dict(
        values=[quantiles[k].tolist() for k in quantiles.columns], align="left"
    )

    table_cells["values"][1] = [
        round(v, round_decimals) for v in table_cells["values"][1]
    ]

    fig.data[0].x = series
    fig.data[1].header = table_header
    fig.data[1].cells = table_cells


def update_size_fig():
    update_quantiles(
        size_fig, trades["size"], "Size", num_decimals(units_per_lot_wid.value)
    )


def update_timestamp_delta_fig():
    time_diffs = trades["timestamp"].diff().dt.total_seconds()
    time_diffs = time_diffs.dropna()

    update_quantiles(timestamp_delta_fig, time_diffs, "Time between Trades (s)", 2)


def update_timestamp_delay_fig():
    update_quantiles(
        timestamp_delay_fig, trades["timestamp_delay"], "Timestamp Delay (s)", 2
    )


def update_bars(fig, categories, values):
    fig.data[0].x = values
    fig.data[0].y = categories


def update_top_makers_fig():
    maker_sum = trades.groupby("maker")["notional"].sum().sort_values(ascending=False)
    maker_sum = maker_sum.head(NUM_TOP_TRADERS)

    top_maker_df = maker_sum.reset_index(name="Notional Volume")
    top_makers_df_out_wid.outputs = []
    top_makers_df_out_wid.append_display_data(HTML(top_maker_df.to_html(index=False)))

    maker_sum.index = maker_sum.index.map(lambda x: f"{x[:4]}...{x[-4:]}")
    # flip order so that largest volume bar is on top
    maker_sum = maker_sum.iloc[::-1]

    update_bars(top_makers_fig, maker_sum.index, maker_sum.values)


def update_top_takers_fig():
    taker_sum = trades.groupby("taker")["notional"].sum().sort_values(ascending=False)
    taker_sum = taker_sum.head(NUM_TOP_TRADERS)

    top_taker_df = taker_sum.reset_index(name="Notional Volume")
    top_takers_df_out_wid.outputs = []
    top_takers_df_out_wid.append_display_data(HTML(top_taker_df.to_html(index=False)))

    taker_sum.index = taker_sum.index.map(lambda x: f"{x[:4]}...{x[-4:]}")
    taker_sum = taker_sum.iloc[::-1]

    update_bars(top_takers_fig, taker_sum.index, taker_sum.values)


def render(*args):
    log_out_wid.outputs = []
    log_out_wid.append_stdout(f"{int(time())}, refreshing every {REFRESH_SEC}s\n")

    try:
        start = time()
        update_trades()
        end = time()
        log_out_wid.append_stdout(f"Fetching trades took {(end - start):.2f}s\n")
    except Exception as e:
        log_out_wid.append_stdout(f"Failed to fetch new trades: {e}\n")
        return

    start = time()

    update_pnl_fig()
    update_inventory_fig()
    update_size_fig()
    update_top_makers_fig()
    update_summary_fig()
    update_timestamp_delta_fig()
    update_timestamp_delay_fig()
    update_top_takers_fig()

    end = time()
    log_out_wid.append_stdout(f"Updating charts took {(end - start):.2f}s\n")

In [ ]:
# Start

In [ ]:
for widget in [
    filename_wid,
    start_time_wid,
    end_time_wid,
    address_wid,
    price_per_tick_wid,
    units_per_lot_wid,
]:
    widget.observe(render, "value")


def on_refresh_click(b):
    render()


refresh_button_wid.on_click(on_refresh_click)

In [ ]:
display(parameters_hbox)
display(spacer)
display(charts_hbox)
display(spacer)
display(trades_out_wid)
display(spacer)
display(makers_takers_hbox)

def thread_func(out_wid):
    while True:
        sleep(REFRESH_SEC)
        render()


thread = threading.Thread(target=thread_func, args=(log_out_wid,))
thread.start()

In [ ]:
# for some reason the voila view wont display the first one or two plotly figures
dummy_series = pd.Series(data=[1], index=[1])
plot_quantiles(dummy_series, bins=2, name="", width=10, height=10).show()
plot_quantiles(dummy_series, bins=2, name="", width=10, height=10).show()